In [7]:
# 필요한 라이브러리 설치
!pip install langchain-community langchain-chroma langchain-openai bs4

# 필요한 라이브러리 임포트
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [9]:
# OpenAI 모델 설정 (gpt-4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# 웹 로더 설정 - 인코딩 설정 추가
loader = WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("css-18vt64m") # 스파르타 블로그의 본문 클래스
        )
    ),
    encoding="utf-8"  # 인코딩 명시적으로 지정. 안하면 깨짐
)


# 문서 로드할 때 인코딩 확인을 위한 출력 추가
docs = loader.load()
print("로드된 문서 내용 미리보기:")
print(docs[0].page_content[:100]) 
# 문서 로드
docs = loader.load()


로드된 문서 내용 미리보기:
스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  738·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적


In [10]:
# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=api_key))

# 검색기(retriever) 생성 - 여기에 검색 개수 증가 코드 적용
# 다양성을 고려한 검색
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 15, "fetch_k": 20}
)
# 사용자 질문에 대한 문서 검색
retrieved_docs = retriever.invoke("ALL-in 코딩 공모전 수상작들을 요약해줘")

# 검색된 문서와 질문을 LLM에 전달
custom_prompt = """당신은 RAG(검색 증강 생성) 시스템입니다. 
다음은 ALL-in 코딩 공모전에 대한 정보입니다:

{context}

요청: {question}

중요 지침:
1. 위 정보를 바탕으로 ALL-in 코딩 공모전의 모든 수상작들을 요약해주세요.
2. 각 수상작의 이름, 등급(대상/우수상/장려상), 개발자, 주요 기능을 포함해야 합니다.
3. 어떤 수상작도 빠뜨리지 말고 모두 포함해주세요.

답변:"""

# 검색된 문서를 포맷팅하여 컨텍스트로 제공
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

user_prompt = {"context": format_docs(retrieved_docs), "question": "ALL-in 코딩 공모전 수상작들을 요약해줘"}
messages = [{"role": "user", "content": custom_prompt.format(**user_prompt)}]
response = llm.invoke(messages)
print(response.content)

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


### ALL-in 코딩 공모전 수상작 요약

1. **대상: Lexi Note**
   - **개발자:** 다나와(김다애, 박나경)
   - **주요 기능:** 단어를 드래그하면 네이버 사전과 번역기와 연동되어 한 번에 필기하고 학습할 수 있는 웹 서비스. 언어 학습에 필요한 할 일 목록과 스케줄 템플릿을 제공하여 효율적인 공부 지원.

2. **우수상: 에코 클래스룸**
   - **개발자:** This is 스파게티!!!(박지성, 김서원, 박범수)
   - **주요 기능:** 교수와 학생 간의 실시간 소통을 지원하며, 학생들이 익명으로 질문하고 의견을 제출할 수 있는 서비스. 교수는 학생들의 이해도를 파악하고, 피드백 및 퀴즈를 통해 수업 속도를 조절할 수 있음.

3. **우수상: 우리집 히어로즈**
   - **개발자:** 인트(배정연, 한지수)
   - **주요 기능:** 자취생들이 자취방에서 겪는 벌레 문제를 해결하기 위한 매칭 서비스. 사용자는 벌레 퇴치 요청을 하고, 신원 보장된 ‘히어로’와 매칭되어 문제를 해결함.

4. **입선: BLOTIE**
   - **개발자:** 블로티(이은주, 한명수, 황준영)
   - **주요 기능:** 외국인 학생과 한국인 학생의 1:1 매칭 기능을 통해 문화와 언어를 자연스럽게Learning 및 교류 지원. 실시간 채팅과 피드 기능 제공.

5. **입선: Crewing**
   - **개발자:** 동학대학운동(김민아, 임경진, 신은혜, 고수)
   - **주요 기능:** 대학생들이 다양한 연합 동아리에 쉽게 가입하고 적절한 동아리를 찾을 수 있도록 지원하는 플랫폼. 동아리 후기와 정보를 제공.

6. **입선: 학교생활 매니저**
   - **개발자:** 아이칼F4(조민제, 이민기, 강건, 박근우)
   - **주요 기능:** 학교 생활을 효율적으로 관리하도록 돕는 앱. 일정 관리, 과제 관리, 성적 예측, 공지사항 업데이트 등의 기능 제공.

이상 ALL-in 코딩 공모전의 수상작들을 요약하였습니다.


In [12]:
# 검색된 문서 내용 확인
print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- 검색된 문서 {i+1} ---")
    print(doc.page_content)
    print("----------------")

검색된 문서 수: 7

--- 검색된 문서 1 ---
스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  738·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어문학을 공부하며 학습 속도도 눈에